In [1]:
from models import YOLOv1
from data.VOC_Dataset import VOC_Dataset
from data import DATA_HOME

from ipdb import set_trace
from torch.utils.data import DataLoader
from numpy import array
from multiprocessing import cpu_count
import random 
import torch
import pandas as pd

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

random.seed(1)
torch.manual_seed(1)

In [ ]:
import platform
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

_voc_root = f"{DATA_HOME}/VOCdevkit/VOC2007"
voc_ds = VOC_Dataset(_voc_root)
train_indices, test_indices = train_test_split(list(range(len(voc_ds))), test_size=0.2, random_state=1)
train_set = Subset(voc_ds, train_indices)
test_set = Subset(voc_ds, test_indices)

BS = 2

def inf_test_gen(data_loader):
    while True:
        for id, batch in enumerate(data_loader):
            yield id, batch
        

def collate_fn(data):
    # output: 7 * 7 * 30
    # S * S * ((x, y, w, h, confidence) * B=2 + C=20)
    # make img batch and label batch
    imgs, labels, classes = zip(*data)
    # inhomegenous shape label, since each image has different number of objects
    # label's dimension: (Batch size, # of objects in each image, 4 coords)
    return torch.tensor(array(imgs), dtype=torch.float), labels, classes
    
if platform.system() == "Windows":
    train_loader = DataLoader(train_set, batch_size=BS, pin_memory=True, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_set, batch_size=BS, pin_memory=True, shuffle=True, collate_fn=collate_fn)
else:
    train_loader = DataLoader(train_set, batch_size=BS, pin_memory=True, shuffle=True, num_workers=4, collate_fn=collate_fn)
    test_loader = DataLoader(test_set, batch_size=BS, pin_memory=True, shuffle=True, num_workers=4, collate_fn=collate_fn)
test_loader = inf_test_gen(test_loader)

class dict:  {'chair': 0, 'bicycle': 1, 'car': 2, 'train': 3, 'bird': 4, 'tvmonitor': 5, 'horse': 6, 'cat': 7, 'diningtable': 8, 'bottle': 9, 'motorbike': 10, 'sofa': 11, 'pottedplant': 12, 'dog': 13, 'aeroplane': 14, 'cow': 15, 'bus': 16, 'boat': 17, 'person': 18, 'sheep': 19}


In [ ]:

yolo = YOLOv1().to(device=device)

S = 7 # num of rows/cols
C = 20 # num of classes
B = 2 # num of bounding boxes

lamba_coord = 5
lamba_noobj = 0.5

In [ ]:
from utils.metrics import IOU
from collections import defaultdict as dd

def yolo_loss(res_mat: torch.tensor, label_mat: list, class_mat: list, loss_df: pd.DataFrame):
    """calcalate batch yolo loss, @param res_mat: (batch_size, B*5+C, S, S)"""
    loss1 = torch.tensor(0.).to(device=device)
    loss2 = torch.tensor(0.).to(device=device)
    loss3 = torch.tensor(0.).to(device=device)
    loss4 = torch.tensor(0.).to(device=device)
    loss5 = torch.tensor(0.).to(device=device)
    # calculate loss for every bounding box in every cell
    for b, batch in enumerate(res_mat):
        # assign labels bbox to cell indices
        stride = 1. / S
        label_inds = dd(list)
        label_class = {}
        for idx, (x, y, w, h) in enumerate(label_mat[b]):
            xi, yi = int(x // stride), int(y // stride)
            label_inds[(xi, yi)].append((x, y, w, h))
            label_class[(x, y, w, h)] = class_mat[b][idx]
        
        # print("label class", label_class, b, batch.shape)
        # iterate cell and calculate loss
        for i in range(batch.shape[1]):
            for j in range(batch.shape[2]):
                cell = batch[:, i, j]

                if (i, j) in label_inds:
                    yprobs = torch.tensor([0.] * C).to(device=device)
                    for x, y, w, h in label_inds[i, j]:
                        for k in range(0, B*5, 5):
                            x_, y_, w_, h_, c_ = cell[k:k+5]
                            loss1 += lamba_coord * ((x-x_) ** 2 + (y-y_) ** 2) # yolo loss term 1
                            loss2 += lamba_coord * ((w - w_.sqrt())**2 + (h ** 0.5 - h_.sqrt())**2) # yolo loss term 2
                            loss3 += (IOU((x, y, w, h), (x_, y_, w_, h_)) - c_) ** 2 # yolo loss term 3
                        yprobs[label_class[x, y, w, h]] = 1.  
                    loss5 += ((yprobs - cell[-C:]) ** 2).sum() # yolo loss term 5
                else:
                    for k in range(4, B*5, 5):
                        _c = cell[k]
                        loss4 += _c ** 2 * lamba_noobj # yolo loss term 4
        # print("xywh: " , x_, y_, w_, h_, c_)
        # print("label: ", x, y, w, h)
    loss_df = pd.concat((loss_df, pd.DataFrame([[loss1.item(), loss2.item(), loss3.item(), loss4.item(), loss5.item()]], columns=loss_df.columns)), ignore_index=True)
    return loss1 + loss2 + loss3 + loss4 + loss5, loss_df
    # print("ret: ", res_mat.shape)
    

In [ ]:
# for i in range(2000):
# _, (batch, labels, classes) = next(enumerate(train_loader))
# res = yolo(batch.cuda())


In [ ]:
# res
# res

In [ ]:
from sys import modules
import os
import torch.optim as optim
import matplotlib.pyplot as plt
import time

save_dir = os.path.expanduser(os.environ["YOLO_MODELS"])

optimizer = optim.Adam(yolo.parameters(), lr=1e-5)
torch.cuda.empty_cache()
train_loss_df = pd.DataFrame(columns=["l1", "l2", "l3", "l4", "l5"])
val_loss_df = pd.DataFrame(columns=["l1", "l2", "l3", "l4", "l5"])
print(f"model save_dir: {save_dir}")
print(f"train_set size: {len(train_set)}, val_set size: {len(test_set)}")
# calculate training time
start_time = time.time()
for epoch in range(10):
    for _id, (batch, labels, classes) in enumerate(train_loader):
        # clear gradient graph
        optimizer.zero_grad()

        # forward-propagate
        res = yolo(batch.to(device=device))
        # print(res.shape, len(labels[2]), classes[2])
        train_loss, train_loss_df = yolo_loss(res, labels, classes, train_loss_df)
        # set_trace

        # back-propagate
        train_loss.backward()
        optimizer.step()

        # calculate validation errors
        _, (batch, labels, classes) = next(test_loader)
        with torch.no_grad():
            res = yolo(batch.to(device=device))
            val_loss, val_loss_df = yolo_loss(res, labels, classes, val_loss_df)
            
        # record losses
        if epoch and epoch % 2 == 0 and _id == 0:

            checkpoint = {
                'model': yolo.state_dict(),
                'optimizer': optimizer.state_dict(),
                'iteration': _id,
                'train_loss': train_loss,
            }
            torch.save(checkpoint, f"{save_dir}/yolov1_{epoch}_{_id}.pth")
        # show time elapse
        if _id and _id % 1000 == 0:
            print(f"Epoch {epoch}, iteration: {_id}, train loss: {train_loss}, val loss: {val_loss}")
            train_loss_df.to_csv(f"{save_dir}/train_loss_latest.csv")
            val_loss_df.to_csv(f"{save_dir}/val_loss_latest.csv")
        
        if _id == 2000:
            # set_trace()
            break
            # end_time = time.time()
            # print("1000 iterations cost: {:.2f} seconds".format(end_time - start_time))
            # start_time = end_time
        # showing the image with labels
        # res_img = Image.fromarray((batch[0] * 255).permute(1, 2, 0).byte().numpy())
        # draw = ImageDraw.Draw(res_img)
        # for pc in labels[0]:
            # draw.rectangle((448*pc[0], 448*pc[1], 448*pc[2], 448*pc[3]), outline="red")
        # res_img.show()

    
    

model save_dir: C:\Users\sxing/YOLO_MODELS
train_set size: 4008, val_set size: 1003
> c:\users\sxing\appdata\local\temp\ipykernel_17800\1623819097.py(55)<module>()

tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[-8.2754e-03, -1.4265e-02, -7.2528e-03,  ..., -3.9035e-03,
           -1.9112e-03, -4.0800e-03],
          [-1.3321e-02, -9.6642e-03, -4.7833e-03,  ...,  4.2232e-03,
           -1.2514e-02, -1.0486e-02],
          [-5.9658e-03, -4.2881e-03,  3.4645e-03,  ..., -1.1782e-02,
            1.1092e-02, -5.4683e-03],
          ...,
          [-9.9226e-03,  1.2295e-02, -3.5795e-03,  ...,  2.2821e-03,
           -1.2040e-02, -1.2504e-02],
          [ 7.6562e-03, -4.0315e-03,  4.4564e-03,  ...,  1.2894e-02,
            4.0858e-03, -1.3988e-02],
          [-7.0184e-04, -1.3607e-02,  5.1404e-03,  ...,  1.2943e-02,
            9.5065e-03, -1.2328e-02]],

         [[-1.8442e-03,  2.2892e-03,  7.7078e-03,  ...,  5.5042e-03,
            1.7918e-03,  1.6850e-02],
          [ 1.35